**1. Import Necessary Libraries**

Run this cell first to import all required libraries.

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose

**2. Generate Synthetic Time Series Data**

This creates a time series dataset with trend, seasonality, and noise.

In [10]:
# Set random seed for reproducibility
np.random.seed(42)

# Generate synthetic time series data for two years (730 days)
dates = pd.date_range(start='2025-01-01', periods=730, freq='D')

# Create the components:
# Trend: a linear increase with a small random walk to add realism.
trend = np.linspace(10, 20, 730) + np.cumsum(np.random.normal(0, 0.05, 730))

# Seasonality: a sine function to simulate yearly repeating patterns.
seasonality = 5 * np.sin(2 * np.pi * np.arange(730) / 365)

# Noise: random fluctuations added to the data.
noise = np.random.normal(0, 0.5, 730)

# Combine the components to create the final time series
data_values = trend + seasonality + noise

# Create a Pandas DataFrame with the synthetic data
df = pd.DataFrame({'date': dates, 'sales': data_values})
df.set_index('date', inplace=True)

**3. Decompose and Plot Time Series Data**

This plots the decomposed generated synthetic sales data.

In [ ]:
# Decompose the time series using an additive model
decomposition = seasonal_decompose(df['sales'], model='additive', period=365, extrapolate_trend='freq')

# Plot the decomposition components
fig = decomposition.plot()
fig.set_size_inches(12, 9)
plt.suptitle(
    "Time Series Decomposition\n"
    "Trend-Cycle Component (Tₜ): Long-term movement\n"
    "Seasonal Component (Sₜ): Regular repeating patterns\n"
    "Remainder Component (Rₜ): Random or irregular fluctuations",
    fontsize=12
)
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

**4.GDP Per Capita UAE**

In [ ]:
file_path = "datasets/Global Economy Indicators.csv"
df = pd.read_csv(file_path)

# Strip whitespace from column names and country names
df.columns = df.columns.str.strip()
df["Country"] = df["Country"].str.strip()

# Filter data for United Arab Emirates
uae_data = df[df["Country"] == "United Arab Emirates"]

# Plot GDP per capita over time
plt.figure(figsize=(10, 5))
plt.plot(uae_data["Year"], uae_data["Per capita GNI"], marker='o', linestyle='-', color='orange')

# Customize plot
plt.xlabel("Year")
plt.ylabel("GDP per Capita (GNI)")
plt.title("GDP per Capita of United Arab Emirates Over Time")
plt.grid(True)

# Show the plot
plt.show()

**5. Box-Cox Transformation**

In [ ]:
# Enable an interactive matplotlib backend.
# If using Jupyter Notebook, you may try:
# %matplotlib widget
# If that doesn't work, try: %matplotlib inline

%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import boxcox, boxcox_normmax
import ipywidgets as widgets
from ipywidgets import interact

# -------------------------------
# Generate Synthetic Time Series Data
# -------------------------------

np.random.seed(42)  # Set random seed for reproducibility

# Create a date range for two years (730 days)
dates = pd.date_range(start='2025-01-01', periods=730, freq='D')

# Generate components of the time series:
# Trend: linear increase with a small random walk for realism
trend = np.linspace(10, 20, 730) + np.cumsum(np.random.normal(0, 0.05, 730))
# Seasonality: yearly seasonality using a sine function
seasonality = 5 * np.sin(2 * np.pi * np.arange(730) / 365)
# Noise: random noise added to the data
noise = np.random.normal(0, 0.5, 730)

# Combine the components
data_values = trend + seasonality + noise

# Ensure that all values are positive (required for Box-Cox transformation)
if (data_values <= 0).any():
    offset = abs(data_values.min()) + 1
    data_values += offset
    print("Data shifted by offset:", offset)

# Create a DataFrame with the synthetic data
df = pd.DataFrame({'date': dates, 'sales': data_values})
df.set_index('date', inplace=True)

# -------------------------------
# Box-Cox Transformation Setup
# -------------------------------

# Choose the data for Box-Cox transformation
data_for_boxcox = df['sales']

# Compute the optimal lambda using maximum likelihood estimation (MLE)
optimal_lambda = boxcox_normmax(data_for_boxcox, method='mle')
print("Optimal lambda (from MLE):", optimal_lambda)

# -------------------------------
# Interactive Box-Cox Transformation Visualization
# -------------------------------

def update_boxcox(lmbda_value):
    """
    Apply the Box-Cox transformation with the provided lambda value and 
    plot the original and transformed series side by side.
    """
    # Apply Box-Cox transformation (using the provided lambda)
    transformed = boxcox(data_for_boxcox, lmbda=lmbda_value)
    
    plt.figure(figsize=(14, 6))
    
    # Plot Original Series
    plt.subplot(1, 2, 1)
    plt.plot(df.index, df['sales'], color='blue', label='Original Sales')
    plt.title("Original Sales Data")
    plt.xlabel("Date")
    plt.ylabel("Sales")
    plt.legend()
    
    # Plot Transformed Series
    plt.subplot(1, 2, 2)
    plt.plot(df.index, transformed, color='orange', label=f"Transformed (λ = {lmbda_value:.2f})")
    plt.title(f"Box-Cox Transformed Sales Data (λ = {lmbda_value:.2f})")
    plt.xlabel("Date")
    plt.ylabel("Transformed Sales")
    plt.legend()
    
    plt.tight_layout()
    plt.show()

# Create an interactive slider for lambda.
interact(update_boxcox, 
         lmbda_value=widgets.FloatSlider(
             value=optimal_lambda, min=-2, max=2, step=0.01, description='Lambda:')
         );

**6.Weighted Moving Average**

In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider
%matplotlib inline

# -------------------------------
# Generate Synthetic Time Series Data
# -------------------------------

np.random.seed(42)  # Set random seed for reproducibility

# Create a date range for two years (730 days)
dates = pd.date_range(start='2025-01-01', periods=730, freq='D')

# Generate components of the time series:
# Trend: linear increase with a small random walk for realism
trend = np.linspace(10, 20, 730) + np.cumsum(np.random.normal(0, 0.05, 730))
# Seasonality: yearly seasonality using a sine function
seasonality = 5 * np.sin(2 * np.pi * np.arange(730) / 365)
# Noise: random noise added to the data
noise = np.random.normal(0, 0.5, 730)

# Combine the components
data_values = trend + seasonality + noise

# Ensure all values are positive (Box-Cox requires positive values, and for our purposes)
if (data_values <= 0).any():
    offset = abs(data_values.min()) + 1
    data_values += offset
    print("Data shifted by offset:", offset)

# Create a DataFrame with the synthetic data
df = pd.DataFrame({'date': dates, 'sales': data_values})
df.set_index('date', inplace=True)

# -------------------------------
# Interactive Moving Average Smoothing with Kernel Visualization
# -------------------------------
# For a 5-term symmetric moving average, we define the kernel as:
# [w, v, c, v, w] where c = 1 - 2*(w+v)
# To ensure nonnegative weights, we require: w + v <= 0.5.

def update_ma(w, v):
    # Check constraint: ensure w + v <= 0.5
    if w + v > 0.5:
        print(f"Invalid weights: Outer + Inner = {w+v:.2f} (must be <= 0.5)")
        return
    
    # Compute the center weight
    c = 1 - 2 * (w + v)
    
    # Create the symmetric kernel
    kernel = np.array([w, v, c, v, w])
    
    # Apply moving average smoothing using convolution in 'valid' mode.
    # A 5-term kernel in 'valid' mode skips the first 2 and last 2 data points.
    smoothed_valid = np.convolve(df['sales'], kernel, mode='valid')
    new_index = df.index[2:-2]  # New index for valid convolution results
    
    # Plot the time series (original and smoothed) and the kernel weights
    fig, axs = plt.subplots(2, 1, figsize=(14, 10))
    
    # Plot 1: Time Series Data
    axs[0].plot(df.index, df['sales'], label="Original Data", color="blue", alpha=0.6)
    axs[0].plot(new_index, smoothed_valid, label="Smoothed Data", color="orange", linewidth=2)
    axs[0].set_title("Moving Average Smoothing with Custom Kernel")
    axs[0].set_xlabel("Date")
    axs[0].set_ylabel("Sales")
    axs[0].legend()
    axs[0].grid(True)
    
    # Plot 2: Kernel Weights as a Bar Chart
    # We'll plot the kernel weights on the x-axis with positions [-2, -1, 0, 1, 2]
    positions = np.array([-2, -1, 0, 1, 2])
    axs[1].bar(positions, kernel, color="green", alpha=0.7)
    for pos, weight in zip(positions, kernel):
        axs[1].text(pos, weight + 0.01, f"{weight:.2f}", ha='center', va='bottom', fontsize=12)
    axs[1].set_title("Kernel Weights: [Outer, Inner, Center, Inner, Outer]")
    axs[1].set_xlabel("Position Relative to Center")
    axs[1].set_ylabel("Weight")
    axs[1].set_ylim(0, max(kernel)*1.2)
    axs[1].grid(True, axis='y')
    
    plt.tight_layout()
    plt.show()

# Create interactive sliders for outer weight (w) and inner weight (v)
interact(update_ma, 
         w=FloatSlider(min=0, max=0.5, step=0.01, value=0.1, description='Outer Weight (w):'),
         v=FloatSlider(min=0, max=0.5, step=0.01, value=0.1, description='Inner Weight (v):'));

**7.Additive and Multiplicative Decomposition**

In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose

# Set random seed for reproducibility
np.random.seed(42)

# -------------------------------
# Generate Synthetic Time Series Data
# -------------------------------

# Create a date range for two years (730 days)
dates = pd.date_range(start='2025-01-01', periods=730, freq='D')

# Generate components of the time series:
# Trend: a linear increase with a small random walk for realism
trend = np.linspace(10, 20, 730) + np.cumsum(np.random.normal(0, 0.05, 730))
# Seasonality: yearly seasonality using a sine function
seasonality = 5 * np.sin(2 * np.pi * np.arange(730) / 365)
# Noise: random noise added to the data
noise = np.random.normal(0, 0.5, 730)

# Combine the components to form the synthetic series
data_values = trend + seasonality + noise

# Ensure all values are positive (required for multiplicative decomposition)
if (data_values <= 0).any():
    offset = abs(data_values.min()) + 1
    data_values += offset
    print("Data shifted by offset:", offset)

# Create a DataFrame with the synthetic data
df = pd.DataFrame({'date': dates, 'value': data_values})
df.set_index('date', inplace=True)

# -------------------------------
# Perform Decomposition
# -------------------------------

# For this demonstration, we use a seasonal period of 365 (yearly seasonality).
# Note: With 730 days of data, we have exactly 2 cycles.
add_decomp = seasonal_decompose(df['value'], model='additive', period=365, extrapolate_trend='freq')
mult_decomp = seasonal_decompose(df['value'], model='multiplicative', period=365, extrapolate_trend='freq')

# -------------------------------
# Plot Additive vs. Multiplicative Decomposition Side by Side
# -------------------------------

# Create a figure with 4 rows (for each component) and 2 columns (additive & multiplicative)
fig, axes = plt.subplots(nrows=4, ncols=2, figsize=(14, 12), sharex=True)

# Titles for the columns
axes[0,0].set_title("Additive Decomposition")
axes[0,1].set_title("Multiplicative Decomposition")

# Row labels for each component
row_labels = ['Observed', 'Trend', 'Seasonal', 'Residual']
components = ['observed', 'trend', 'seasonal', 'resid']

# Plot each component for both decomposition methods
for i, comp in enumerate(components):
    # Additive decomposition plot
    axes[i, 0].plot(add_decomp.observed.index, getattr(add_decomp, comp), color='blue')
    axes[i, 0].set_ylabel(row_labels[i])
    
    # Multiplicative decomposition plot
    axes[i, 1].plot(mult_decomp.observed.index, getattr(mult_decomp, comp), color='red')
    axes[i, 1].set_ylabel(row_labels[i])

# Set x-axis label on the bottom row
axes[3,0].set_xlabel("Date")
axes[3,1].set_xlabel("Date")

plt.suptitle("Additive vs. Multiplicative Decomposition", fontsize=16)
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

**8.STL Decomposition**

In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import STL, seasonal_decompose
import ipywidgets as widgets
from ipywidgets import interact
%matplotlib inline

# -------------------------------
# Generate Synthetic Time Series Data
# -------------------------------

np.random.seed(42)  # For reproducibility

# Create a date range for two years (730 days)
dates = pd.date_range(start='2025-01-01', periods=730, freq='D')

# Generate synthetic components:
# Trend: Linear increase with a small random walk for realism
trend = np.linspace(10, 20, 730) + np.cumsum(np.random.normal(0, 0.05, 730))
# Seasonality: Yearly seasonality using a sine function
seasonality = 5 * np.sin(2 * np.pi * np.arange(730) / 365)
# Noise: Random noise added to the data
noise = np.random.normal(0, 0.5, 730)

# Combine components to form the synthetic series
data_values = trend + seasonality + noise

# Ensure all values are positive (required for multiplicative methods)
if (data_values <= 0).any():
    offset = abs(data_values.min()) + 1
    data_values += offset
    print("Data shifted by offset:", offset)

# Create a DataFrame with the synthetic data
df = pd.DataFrame({'date': dates, 'sales': data_values})
df.set_index('date', inplace=True)

# -------------------------------
# Interactive Decomposition Comparison
# -------------------------------
def update_decompositions(trend_window, seasonal_window):
    """
    Update and plot STL and classical additive decompositions for the synthetic series,
    using interactive sliders for the STL trend and seasonal window parameters.
    For daily data with period=365, trend_window must be > 365.
    """
    # Ensure the provided parameters are integers and odd.
    trend_window = int(trend_window)
    seasonal_window = int(seasonal_window)
    if trend_window % 2 == 0:
        trend_window += 1
    if seasonal_window % 2 == 0:
        seasonal_window += 1

    # Print the parameters for clarity.
    print(f"Using STL parameters: trend window = {trend_window}, seasonal window = {seasonal_window}")
    
    # Perform STL decomposition with the chosen parameters.
    stl = STL(df['sales'], period=365, trend=trend_window, seasonal=seasonal_window, robust=True)
    stl_result = stl.fit()
    
    # Perform classical additive decomposition using seasonal_decompose.
    classical_result = seasonal_decompose(df['sales'], model='additive', period=365, extrapolate_trend='freq')
    
    # Create a figure with 4 rows (for each component) and 2 columns (STL vs. Classical)
    fig, axes = plt.subplots(nrows=4, ncols=2, figsize=(14, 12), sharex=True)
    plt.subplots_adjust(hspace=0.4)
    
    # Titles for each column
    axes[0, 0].set_title(f"STL Decomposition\n(trend window = {trend_window}, seasonal window = {seasonal_window})")
    axes[0, 1].set_title("Classical Decomposition")
    
    # Define component labels and mapping for STL
    components = ['Observed', 'Trend', 'Seasonal', 'Residual']
    stl_components = {
        'Observed': stl_result.observed,
        'Trend': stl_result.trend,
        'Seasonal': stl_result.seasonal,
        'Residual': stl_result.resid
    }
    
    # For classical decomposition, the attributes are: observed, trend, seasonal, resid
    classical_components = {
        'Observed': classical_result.observed,
        'Trend': classical_result.trend,
        'Seasonal': classical_result.seasonal,
        'Residual': classical_result.resid
    }
    
    # Plot each component for both decompositions
    for i, comp in enumerate(components):
        # Plot STL decomposition (left column)
        axes[i, 0].plot(stl_components[comp].index, stl_components[comp], label=comp, color=f"C{i}")
        axes[i, 0].set_ylabel(comp)
        axes[i, 0].legend(loc='upper left')
        
        # Plot Classical decomposition (right column)
        axes[i, 1].plot(classical_components[comp].index, classical_components[comp], label=comp, color=f"C{i}")
        axes[i, 1].set_ylabel(comp)
        axes[i, 1].legend(loc='upper left')
    
    # Set x-axis labels on the bottom row for both columns
    axes[3, 0].set_xlabel("Date")
    axes[3, 1].set_xlabel("Date")
    
    fig.suptitle("Comparison of STL vs. Classical Additive Decomposition", fontsize=16)
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.show()

# Create interactive sliders:
# For daily data with period=365, the STL trend window must be greater than 365.
# We set the trend window slider from 367 to 701 (odd values only).
# The seasonal window slider can be from 3 to 31.
interact(update_decompositions, 
         trend_window=widgets.IntSlider(min=367, max=701, step=2, value=367, description='Trend Window'),
         seasonal_window=widgets.IntSlider(min=3, max=31, step=2, value=11, description='Seasonal Window'));

**9.Seasonally Adjusted Data**

In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose

%matplotlib inline

# =============================================================================
# 6. Seasonally Adjusted Data
#
# Definition:
# Once the seasonal component is removed from the original data, the remaining
# values are called seasonally adjusted data.
#   Additive: yₜ − Sₜ
#   Multiplicative: yₜ / Sₜ
#
# Usage:
# Seasonally adjusted series are often used when the focus is on understanding
# the underlying trend and cyclic behavior without the distraction of regular
# seasonal effects.
# =============================================================================

# -------------------------------
# Part A: Additive Series Example
# -------------------------------

# Generate synthetic additive series:
np.random.seed(42)  # For reproducibility
dates = pd.date_range(start='2025-01-01', periods=730, freq='D')

# Components for additive series
trend_add = np.linspace(50, 60, 730)                           # Linear trend
seasonal_add = 10 * np.sin(2 * np.pi * np.arange(730) / 365)     # Seasonal (sine wave) component
noise_add = np.random.normal(0, 2, 730)                          # Random noise

# Create additive series: y = trend + seasonal + noise
y_add = trend_add + seasonal_add + noise_add

# Create DataFrame for additive series
df_add = pd.DataFrame({'date': dates, 'y': y_add})
df_add.set_index('date', inplace=True)

# Perform additive decomposition (using a seasonal period of 365 days)
decomp_add = seasonal_decompose(df_add['y'], model='additive', period=365, extrapolate_trend='freq')

# Compute seasonally adjusted data: Remove the seasonal component
seasonally_adjusted_add = decomp_add.observed - decomp_add.seasonal

# -------------------------------
# Part B: Multiplicative Series Example
# -------------------------------

# For a multiplicative series, the seasonal effect is proportional to the level.
# Generate synthetic multiplicative series:
np.random.seed(42)
# Trend remains similar
trend_mult = np.linspace(50, 60, 730)
# Seasonal multiplicative component: oscillates around 1 (e.g., 1 ± 0.2)
seasonal_mult = 1 + 0.2 * np.sin(2 * np.pi * np.arange(730) / 365)
# Multiplicative noise around 1 (small variation)
noise_mult = np.random.normal(1, 0.05, 730)

# Create multiplicative series: y = trend * seasonal * noise
y_mult = trend_mult * seasonal_mult * noise_mult

# Create DataFrame for multiplicative series
df_mult = pd.DataFrame({'date': dates, 'y': y_mult})
df_mult.set_index('date', inplace=True)

# Perform multiplicative decomposition
decomp_mult = seasonal_decompose(df_mult['y'], model='multiplicative', period=365, extrapolate_trend='freq')

# Compute seasonally adjusted data: Divide out the seasonal component
seasonally_adjusted_mult = decomp_mult.observed / decomp_mult.seasonal

# -------------------------------
# Plotting the Results Side by Side
# -------------------------------

fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(14, 10), sharex=True)

# Plot for the Additive Series
axes[0].plot(df_add.index, df_add['y'], label='Original Additive Series', color='blue', alpha=0.6)
axes[0].plot(df_add.index, seasonally_adjusted_add, label='Seasonally Adjusted (y - Sₜ)', color='orange', linewidth=2)
axes[0].set_title("Additive Series: Original vs. Seasonally Adjusted")
axes[0].set_ylabel("Value")
axes[0].legend()
axes[0].grid(True)

# Plot for the Multiplicative Series
axes[1].plot(df_mult.index, df_mult['y'], label='Original Multiplicative Series', color='green', alpha=0.6)
axes[1].plot(df_mult.index, seasonally_adjusted_mult, label='Seasonally Adjusted (y / Sₜ)', color='red', linewidth=2)
axes[1].set_title("Multiplicative Series: Original vs. Seasonally Adjusted")
axes[1].set_xlabel("Date")
axes[1].set_ylabel("Value")
axes[1].legend()
axes[1].grid(True)

plt.tight_layout()
plt.show()

## **How to Use This Notebook**

1.	Copy each section into separate cells in an IPython notebook.
2.	Run them sequentially to generate and explore time series data.
3.	Modify the dataset generation (e.g., change period=365 in decomposition for different frequencies).
4.	Save the data for later forecasting model training.

This setup provides a hands-on approach to understanding time series forecasting while using LLMs to generate relevant Python code. 🚀